# Introduction to Debug Tracing for Side-Channel Analysis

Setup depends on the trace platform, target platform, and trace interface, so make the correct choices below.

In addition to ChipWhisperer capture platform (pro or lite), you need either:
* CW305
* CW610 (PhyWhisperer); in this case you also need one of:
    * CW308 multi-target board with the MK82F target
    * CW308 with an STM32 target
    * CWLITEARM

This notebook tries to highlight some of the many things that can be done with trace, and so it is not meant to be run "straight through". Read the instructions carefully!

In [ ]:
TRACE_PLATFORM = 'CW610' # AKA PhyWhisperer
#TRACE_PLATFORM = 'CW305' # CW305 FPGA target board

#PLATFORM = 'CWLITEARM'
PLATFORM = 'CW308_STM32F3'
#PLATFORM = 'CW308_K82F'

#TRACE_INTERFACE = 'parallel'
TRACE_INTERFACE = 'swo'

In [ ]:
import chipwhisperer as cw
from chipwhisperer.capture.trace.TraceWhisperer import TraceWhisperer

In [ ]:
##### TODO: point to standard bitfile and defines ########
defines = ['../hardware/CW305_DesignStart/hdl/defines_trace.v', '../hardware/phywhisperer/software/phywhisperer/firmware/defines_pw.v']

In [ ]:
# platform setup:
if TRACE_PLATFORM == 'CW610':
    SCOPETYPE = 'OPENADC'
    %run "Helper_Scripts/Setup_Generic.ipynb"
    trace = TraceWhisperer(target, force_bitfile=False, defines_files=defines)
    # on this platform, minimum trace frequency is 10 MHz, so minimum target frequency is twice that; increase baud rate accordingly:
    #scope.clock.clkgen_freq = 20e6
    #target.baud = 104000
    scope.clock.adc_src = "clkgen_x4"
    if PLATFORM == 'CWLITEARM':
        scope.adc.samples = 24400
    else:
        scope.adc.samples = 30000
    scope.gain.setGain(20)

else:
    %run "Helper_Scripts/Setup_CW305_DST.ipynb"
    scope.adc.samples = 35000
    trace = TraceWhisperer(target, scope, defines_files=defines)

In [ ]:
help(trace)

In [ ]:
trace._naeusb.readFwVersion()

In [ ]:
# TODO: set gain appropriately for each target/platform
if TRACE_PLATFORM == 'CW305':
    scope.gain.setGain(30)
elif PLATFORM == 'CW308_K82F':
    scope.gain.setGain(20)
elif PLATFORM == 'CW308_STM32F3':
    scope.gain.setGain(25)
elif PLATFORM == 'CWLITEARM':
    scope.gain.setGain(25)

### Program STM32 target:

If you're using the K82 target, you'll need an external programmer.

In [ ]:
if (PLATFORM == 'CW308_STM32F3') or (PLATFORM == 'CWLITEARM'):
    fw_path = '../../cw_dev3/hardware/victims/firmware/simpleserial-trace/simpleserial-trace-CW308_STM32F3.hex'
    prog = cw.programmers.STM32FProgrammer
    cw.program_target(scope, prog, fw_path)

In [ ]:
# required after programming some targets:
def target_reset():
    if TRACE_PLATFORM == 'CW610':
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

In [ ]:
target_reset()

In [ ]:
# target info and buildtimes:
print(trace.get_target_name())
print(trace.get_fw_buildtime())
print(trace.get_fpga_buildtime())

In [ ]:
######## TODO: remove ##########
# set correct board revision:
trace.fpga_write(trace.REG_BOARD_REV, [4])

### Set clock and baud rate for SWO mode:

*(skip if using the parallel trace port)*

There are lots of knobs in setting the SWO bit rate. Sensible default settings are given here, but if you want to modify them, you'll first have to understand what the knobs do.

First, there is the target's clock, which is sourced by the ChipWhisperer's `clkgen`.

Then, the target's `TPI.ACPR` register, which defines how many clock cycles per SWO bit. The SWO baud rate is equal to the target clock divided by (ACPR + 1). So, setting this register to zero means an SWO bit on every target clock cycle.

Finally, there is the CW610's internal UART, which is running at 192 MHz: we need to define how many of *its* clock cycles each SWO bit lasts. Since these bits traverse jumper cables, it's best to be conservative here. Or run the risk of mis-sampling the trace data.

In [ ]:
# These are the base constants, do not modify:
base_target_clock = 7.384e6
base_baud = 38400
usb_clock = 96e6
uart_clock = usb_clock * 2

In [ ]:
# These are your variables. Stick with the defaults, or, if you know what you're doing, edit carefully:
ACPR = 0
swo_div = 8

In [ ]:
new_target_clock = int(uart_clock / (swo_div * (ACPR+1)))

In [ ]:
# now carry out the changes:
new_target_clock = int(uart_clock / (swo_div * (ACPR+1)))
scope.clock.clkgen_freq = new_target_clock
target.baud = int(base_baud * (new_target_clock/base_target_clock))
trace.set_reg('TPI_ACPR', '%08x' % ACPR)

In [ ]:
scope.clock.reset_adc()
assert (scope.clock.adc_locked), "ADC failed to lock"

#### Check that the target is alive.
If `get_fw_buildtime()` produces no output, the target is unresponsive; it may simply require a reset.

In [ ]:
print(trace.get_fw_buildtime())

In [ ]:
# reset if needed
target_reset()

### Set clock and baud rate for parallel trace mode:
On the CW610 platform, the target clock must be at least 20 MHz, otherwise the CW610's PLL may fail to lock:

In [ ]:
if TRACE_INTERFACE == 'parallel' and TRACE_PLATFORM == 'CW610':
    scope.clock.clkgen_freq = 20e6
    target.baud = 104000

## Set trace or SWO operation mode:

Arm processors which support JTAG and SWD come out of reset in JTAG mode. In order to get trace data out of the SWO pin, we need to switch it over to SWD mode.

The `jtag_to_swd()` call below runs a special sequence on the TMS and TCK pins to do this switchover. However, different processors (like for example the STM32) may have *additional* requirements to enable the SWO pin. The `simpleserial-trace` firmware handles this for the STM32.

Long story short: this will work for our STM32 and K82 targets; other targets may need additional setup. One sure-fire way to get a target into SWD mode is to use an external debugger. In that case, do not call `jtag_to_swd()`, as this could result in contention on the TMS/TCK pins.

In [ ]:
if TRACE_INTERFACE == 'parallel':
    if TRACE_PLATFORM == 'CW610':
        print("*** Don't forget the jumper cables from the target's trace pins to the PhyWhisperer D[4:7] and CK pins ***")
    trace.set_mode('trace')
else:
    print("*** Don't forget the jumper cables from the target's TMS/TCK/TDO pins to the PhyWhisperer D0/D1/D2 pins ***")
    trace.set_mode('swo', swo_div=8)
    trace.jtag_to_swd()

trace.check_clocks()

### Trigger trace capture from target FW:

In [ ]:
trace.use_soft_trigger()

### What to capture:
There are two trace capture modes:
1. Raw mode captures the complete raw trace data.
2. Non-raw mode captures only matching rule IDs. To use this, set up some pattern match rules (see below); only the ID of the matching rule will be captured.

In [ ]:
trace.fpga_write(trace.REG_CAPTURE_RAW, [0])

In [ ]:
##### TODO: remove #########
#trace.fpga_write(trace.REG_PATTERN_ENABLE, [0])

### Alternatively, set a pattern matching rule and capture only rule match IDs:

In [ ]:
trace.fpga_write(trace.REG_CAPTURE_RAW, [0])

# match on any PC match (isync) trace packet:
trace.set_pattern_match(0, [3, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])

# match on anything:
#trace.set_pattern_match(0, [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0])

# enable matching rule:
trace.fpga_write(trace.REG_PATTERN_ENABLE, [1])

### Optionally, a pattern matching rule can be used to trigger trace capture (instead of the target FW soft trigger):

Be aware that this is not a stable trigger, expect jitter up to 6 clock cycles.

In [ ]:
trace.use_trace_trigger(rule=0)

### How long to capture for:
By default, trace data is captured while the target's trigger line is high. This is probably what you want to do (unless your target doesn't drive the trigger line).

Alternatively, you can manually specify how many cycles or events the capture should last, by setting `REG_CAPTURE_LEN`.

What's counted? If `REG_COUNT_WRITES` is set, then `REG_CAPTURE_LEN` FIFO writes (trace bytes and timestamps) captured.

Otherwise, the capture is active for `REG_CAPTURE_LEN` clock cycles.

If `REG_CAPTURE_LEN` is set to zero, then the capture runs until the internal buffer is filled.

In [ ]:
trace.fpga_write(trace.REG_CAPTURE_WHILE_TRIG, [0])
trace.fpga_write(trace.REG_COUNT_WRITES, [0])
trace.fpga_write(trace.REG_CAPTURE_LEN, int.to_bytes(40000, length=4, byteorder='little'))

### Set PC addresses to match on:
Let's use the start of the `SubBytes()` and `MixColumns()` functions.

This will set the target's `DWT.COMP0`, `DWT.COMP1`, and `ETM.TEEVR` registers.

In [ ]:
if TRACE_PLATFORM == 'CW305':
    trace.set_isync_matches(addr0=0x3bc0, addr1=0x3aa8, match='both')
elif PLATFORM == 'CWLITEARM' or PLATFORM == 'CW308_STM32F3':
    trace.set_isync_matches(addr0=0x08001728, addr1=0x08001736, match='both')
elif PLATFORM == 'CW308_K82F':
    trace.set_isync_matches(addr0=0x1d70, addr1=0x1d7c, match='both')

In [ ]:
trace.get_reg('DWT_COMP0'), trace.get_reg('DWT_COMP1')

### Enable or disable periodic PC sampling:

In [ ]:
trace.set_periodic_pc_sampling(enable=0)

# Capture power and debug trace:

In [ ]:
if TRACE_PLATFORM == 'CW610':
    print("*** Don't forget the jumper cable from CW308 GPIO4/TRIG pin to PhyWhisperer PC pin on side connector! ***")

In [ ]:
if TRACE_PLATFORM == 'CW610':
    sstarget = target
else:
    sstarget = trace._ss

In [ ]:
# force resynchronization, ensure we are sync'd:
if TRACE_INTERFACE == 'parallel':
    trace.resync()

In [ ]:
# arm trace sniffer:
trace.arm_trace()

In [ ]:
from tqdm import tnrange
import numpy as np

ktp = cw.ktp.Basic()

powertraces = []
num_traces = 1

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()  # manual creation of a key, text pair can be substituted here
    powertrace = cw.capture_trace(scope, sstarget, text, key)
    if powertrace is None:
        continue
    powertraces.append(powertrace)

#Convert traces to numpy arrays
trace_array = np.asarray([trace.wave for trace in powertraces])  # if you prefer to work with numpy array for number crunching
textin_array = np.asarray([trace.textin for trace in powertraces])
known_keys = np.asarray([trace.key for trace in powertraces])  # for fixed key, these keys are all the same

### Read the raw trace data:

In [ ]:
raw = trace.read_capture_data()
len(raw)

### If we captured raw data, parse out raw 'frames' from it:
This will *not* work if you used `trace.fpga_write(trace.REG_CAPTURE_RAW, [0])`!

When using the parallel trace port, the sync frames are used as frame delimiters.

In [ ]:
if TRACE_INTERFACE == 'parallel':
    frames = trace.get_raw_trace_packets(raw, removesyncs=True, verbose=True)
else:
    frames = trace.get_raw_trace_packets(raw, removesyncs=False, verbose=True)

#### Each entry of `frames` contains a timestamp (# of clock cycles elapsed since trigger) and a payload:
This only works in parallel trace mode because we can use the sync frames as frame delimiters. But fear not, Orbuculum can parse the trace data either way (a few cells down).

In [ ]:
if TRACE_INTERFACE == 'parallel':
    frames[:3]

### If we captured matching rule events, this will list timestamped matching rule IDs:
This will *not* work if you used `trace.fpga_write(trace.REG_CAPTURE_RAW, [1])`!

In [ ]:
times = trace.get_rule_match_times(raw, rawtimes=False, verbose=True)

In [ ]:
len(times)

# Parse the raw trace data with Orbuculum:
For the case where `REG_CAPTURE_RAW = 1` only.

Orbuculum allows you to make sense out of the cryptic TPIU-encoded trace data. It can be installed from: https://github.com/orbcode/orbuculum

In [ ]:
# first, write out the raw trace data to a file:
trace.write_raw_capture(frames, 'raw.bin')

In [ ]:
# change the path to where the orbuculum executable resides on your own system:

In [ ]:
%%bash
/home/jp/github/orbuculum/ofiles/orbuculum -t -f raw.bin -P -e
cat hwevent

Refer to Orbuculum documentation for more information, but for the example shown here you'll get two types of entries out of Orbuculum:
1. Starts with '2': periodic PC sample; last field is the PC value
2. Starts with '8': Isync match; last field is the PC value

The middle field is the timestamp inferred by Orbuculum, which is inaccurate here since TraceWhisperer strips out most of the sync frames for storage efficiency and records its own timestamps instead.

# Plotting Example
For the code below, go back above and re-run a trace capture with non-raw capture mode, using one or two PC addresses that are of interest to you.
Skip over the Orbuculum cells since we aren't capturing raw trace packets.

The default PC match values, for the target executable in the repository, are the start of the `SubBytes()` and `MixColumns()` functions.

The code below overlays black vertical lines on top of the power trace, for each rule match event.

Note that 18 matches are obtained (not 20) because the last round uses a different code path.

In order to overlay the power and debug trace data, we must match their timescales. `multiplier` expresses the ratio of the power trace sampling rate to the debug trace sampling rate. First, we account for the x1 or x4 power trace sampling rate.

Then, we account for the debug trace sampling rate. In parallel trace mode this is straightforward -- the debug trace rate is equal to the target processor speed.

In SWO mode, things are a bit more complicated because the debug trace sampling rate and target clock rate can be different.

See https://github.com/newaetech/DesignStartTrace/blob/master/hardware/tracewhisperer/clocks.md for details on what's happening here.

In [ ]:
if scope.clock.adc_src == 'clkgen_x4' or scope.clock.adc_src == 'extclk_x4':
    multiplier = 4
else:
    multiplier = 1

if TRACE_INTERFACE == 'swo':
    multiplier /= (usb_clock/new_target_clock)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
p = figure(plot_width=1200)

xrange = range(len(powertraces[0].wave))
p.line(xrange, powertraces[0].wave, line_color="red")

vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=2))
p.renderers.extend(vlines)

In [ ]:
show(p)